# Load Data

In [1]:
# Importing libraries used on this project

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob as glob
from pycaret.regression import *

In [2]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [3]:
# Loading all csv files into one object called "files"

files = glob.glob('./data_StatsCan/en_climate_daily_NS_*.csv')
files

# Concatenated all csv files loaded into one DataFrame for analysis

df = pd.concat([pd.read_csv(f, low_memory=False) for f in files], ignore_index=True)
df.head()

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),...,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
0,-63.5,44.88,HALIFAX STANFIELD INT'L A,8202250,1960-01-01,1960,1,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-63.5,44.88,HALIFAX STANFIELD INT'L A,8202250,1960-01-02,1960,1,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-63.5,44.88,HALIFAX STANFIELD INT'L A,8202250,1960-01-03,1960,1,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-63.5,44.88,HALIFAX STANFIELD INT'L A,8202250,1960-01-04,1960,1,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-63.5,44.88,HALIFAX STANFIELD INT'L A,8202250,1960-01-05,1960,1,5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This project is going to focus on a few parameters of weather data from Halifax Stanfield Aiport. The main parameter studied is the evolution of mean temperature along the years and how climate is changing. 

Another parameter of interest, which is somehow related to climate change is the Heating Degree-Days (HDD). HDD is used to calculate heat demand and insulation requirements for buildings in the construction industry.

HDD = Heating degree-days for a given day are the number of degrees Celsius that the mean temperature is below 18 °C. If the temperature is equal to or greater than 18 °C, then the number will be zero. For example, a day with a mean temperature of 15.5 °C has 2.5 heating degree-days; a day with a mean temperature of 20.5 °C has zero heating degree-days. Heating degree-days are used primarily to estimate the heating requirements of buildings.

[Climate Weather Canada](https://climate.weather.gc.ca/glossary_e.html#hdd)

# EDA and Data Preparation

In [4]:
# Checking for the shape of data
df.shape

(23742, 31)

In [5]:
# Checking for nulls
df.isna().sum()

Longitude (x)                    0
Latitude (y)                     0
Station Name                     0
Climate ID                       0
Date/Time                        0
Year                             0
Month                            0
Day                              0
Data Quality                 23742
Max Temp (°C)                  935
Max Temp Flag                23657
Min Temp (°C)                  937
Min Temp Flag                23655
Mean Temp (°C)                 945
Mean Temp Flag               23655
Heat Deg Days (°C)             945
Heat Deg Days Flag           23655
Cool Deg Days (°C)             945
Cool Deg Days Flag           23655
Total Rain (mm)               1000
Total Rain Flag              20720
Total Snow (cm)                918
Total Snow Flag              21058
Total Precip (mm)              964
Total Precip Flag            19235
Snow on Grnd (cm)             3733
Snow on Grnd Flag            22052
Dir of Max Gust (10s deg)     8371
Dir of Max Gust Flag

In [6]:
# listing DataFrame columns
df.columns

Index(['Longitude (x)', 'Latitude (y)', 'Station Name', 'Climate ID',
       'Date/Time', 'Year', 'Month', 'Day', 'Data Quality', 'Max Temp (°C)',
       'Max Temp Flag', 'Min Temp (°C)', 'Min Temp Flag', 'Mean Temp (°C)',
       'Mean Temp Flag', 'Heat Deg Days (°C)', 'Heat Deg Days Flag',
       'Cool Deg Days (°C)', 'Cool Deg Days Flag', 'Total Rain (mm)',
       'Total Rain Flag', 'Total Snow (cm)', 'Total Snow Flag',
       'Total Precip (mm)', 'Total Precip Flag', 'Snow on Grnd (cm)',
       'Snow on Grnd Flag', 'Dir of Max Gust (10s deg)',
       'Dir of Max Gust Flag', 'Spd of Max Gust (km/h)',
       'Spd of Max Gust Flag'],
      dtype='object')

In [7]:
# Droping columns that are not going to be used on this project
df = df.drop(['Longitude (x)', 'Latitude (y)', 'Station Name', 'Climate ID','Max Temp Flag','Heat Deg Days Flag','Min Temp Flag',
              'Data Quality','Mean Temp Flag', 'Cool Deg Days (°C)', 'Cool Deg Days Flag', 'Total Rain (mm)',
       'Total Rain Flag', 'Total Snow (cm)', 'Total Snow Flag',
       'Total Precip (mm)', 'Total Precip Flag', 'Snow on Grnd (cm)',
       'Snow on Grnd Flag', 'Dir of Max Gust (10s deg)',
       'Dir of Max Gust Flag', 'Spd of Max Gust (km/h)',
       'Spd of Max Gust Flag'], axis=1)
df

,Date/Time,Year,Month,Day,Max Temp (°C),Min Temp (°C),Mean Temp (°C),Heat Deg Days (°C)
0,1960-01-01,1960,1,1,NaN,NaN,NaN,NaN
1,1960-01-02,1960,1,2,NaN,NaN,NaN,NaN
2,1960-01-03,1960,1,3,NaN,NaN,NaN,NaN
3,1960-01-04,1960,1,4,NaN,NaN,NaN,NaN
4,1960-01-05,1960,1,5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
23737,2023-12-27,2023,12,27,NaN,NaN,NaN,NaN
23738,2023-12-28,2023,12,28,NaN,NaN,NaN,NaN
23739,2023-12-29,2023,12,29,NaN,NaN,NaN,NaN
23740,2023-12-30,2023,12,30,NaN,NaN,NaN,NaN


In [8]:
# Verifying for nulls on the remaining data that will be used
df.isna().sum()

Date/Time               0
Year                    0
Month                   0
Day                     0
Max Temp (°C)         935
Min Temp (°C)         937
Mean Temp (°C)        945
Heat Deg Days (°C)    945
dtype: int64

In [9]:
# Droping nulls from the data
df = df.dropna()
df.isna().sum()

Date/Time             0
Year                  0
Month                 0
Day                   0
Max Temp (°C)         0
Min Temp (°C)         0
Mean Temp (°C)        0
Heat Deg Days (°C)    0
dtype: int64

In [10]:
# Verifying main statistics on the data. Notice that there are lots os missing data on 1960 compared to other years (only 184 counts). 
# Also, there is only 92 rows in 2023 because it is the current year.
df.groupby('Year')[['Mean Temp (°C)']].describe().T

Year                        1960        1961        1962        1963  \
Mean Temp (°C) count  184.000000  365.000000  365.000000  365.000000   
               mean    10.370652    6.421644    5.543836    5.694795   
               std      8.648011   10.085186    9.343123    9.756776   
               min    -12.800000  -19.500000  -19.200000  -17.500000   
               25%      3.100000   -1.100000   -0.600000   -1.700000   
               50%     11.550000    6.400000    6.400000    6.400000   
               75%     18.100000   15.600000   13.400000   14.200000   
               max     24.800000   25.900000   21.700000   27.000000   

Year                        1964        1965        1966        1967  \
Mean Temp (°C) count  366.000000  365.000000  365.000000  365.000000   
               mean     5.698087    5.341644    6.708767    5.780822   
               std      8.707344    9.454167    8.728190   10.080103   
               min    -13.400000  -19.700000  -21.100000  -20.000000   
               25%     -1.400000   -2.200000   -0.300000   -2.300000   
               50%      6.150000    5.000000    7.000000    5.600000   
               75%     13.400000   13.900000   14.700000   14.700000   
               max     22.200000   22.500000   23.100000   22.300000   

Year                        1968        1969  ...        2014        2015  \
Mean Temp (°C) count  366.000000  365.000000  ...  365.000000  365.000000   
               mean     6.163934    6.977534  ...    7.099178    6.476438   
               std      9.975271    8.769505  ...    9.701582   10.461173   
               min    -21.700000  -14.800000  ...  -18.600000  -16.600000   
               25%     -0.900000   -0.600000  ...    0.000000   -2.200000   
               50%      7.250000    7.500000  ...    7.700000    6.700000   
               75%     14.700000   14.500000  ...   15.000000   15.900000   
               max     25.000000   24.500000  ...   22.700000   23.400000   

Year                        2016        2017        2018        2019  \
Mean Temp (°C) count  363.000000  359.000000  357.000000  360.000000   
               mean     7.344077    7.529805    7.364706    6.755833   
               std      9.337780    9.598454    9.660297    9.506765   
               min    -15.800000  -14.900000  -14.100000  -12.600000   
               25%      0.000000   -0.300000    0.100000   -1.500000   
               50%      7.600000    8.400000    7.000000    6.750000   
               75%     15.250000   16.200000   15.300000   14.700000   
               max     23.500000   23.500000   25.400000   26.000000   

Year                        2020        2021        2022       2023  
Mean Temp (°C) count  349.000000  340.000000  363.000000  92.000000  
               mean     7.849570    8.467059    8.166942  -2.103261  
               std      9.286428    9.109338    9.569401   4.900178  
               min    -15.000000  -10.600000  -16.000000 -20.800000  
               25%      0.400000    0.675000    1.250000  -4.525000  
               50%      7.200000    8.800000    8.400000  -1.300000  
               75%     15.400000   16.600000   16.700000   1.100000  
               max     24.800000   25.200000   25.500000   7.400000  

[8 rows x 64 columns]

In [11]:
# To avoid statistical influence on mean temperatures from missing data it will be droped data from 1960 and 2023.
df = df[(df['Year']!=1960) & (df['Year']!=2023)]

In [12]:
# Data is clean now and ready for evaluation
df.isna().sum()

Date/Time             0
Year                  0
Month                 0
Day                   0
Max Temp (°C)         0
Min Temp (°C)         0
Mean Temp (°C)        0
Heat Deg Days (°C)    0
dtype: int64

In [13]:
# This is the shape of the DataFrame
df.shape

(22521, 8)